In [1]:
# Import necessary packages
import numpy as np
import pandas as pd
import ruamel.yaml as yaml
import os, sys ,re

# Define yaml loader
class SafeLoaderIgnoreUnknown(yaml.SafeLoader):
    def ignore_unknown(self, node):
        return None 
SafeLoaderIgnoreUnknown.add_constructor(None, SafeLoaderIgnoreUnknown.ignore_unknown)

# Choose the runs to include in analysis
#runs_to_analyze = [ {"config":"naive_bs_as_n2","nickname":"naive_dqn_bs_as_n2_2000_v1"}]
runs_to_analyze = [ {"config":"naive_bs_as_n2","nickname":"naive_dqn_bs_as_n2_2000_v1"},
                    {"config":"naive_bs_as_n4","nickname":"naive_dqn_bs_as_n4_2000_v1"},
                    {"config":"naive_bs_as_n6","nickname":"naive_dqn_bs_as_n6_2000_v1"},
                    {"config":"naive_bs_as_n8","nickname":"naive_dqn_bs_as_n8_2000_v1"}]

# Set relative path to data
rule_runs_path = "../active/outputs/rule_runs/"

In [2]:
# Function takes reward value per episode and calculates total moves, move accuracy, and whether the board was cleared
def move_calculator(row,obj):
    # Given a dataframe row, grab the reward value
    reward = row['reward']
    # Rewards default to negative
    err = round(-1*reward)
    # Separate cases for board being not cleared vs. cleared
    # Episodes limited to 100 moves and board begins with fixed number of pieces, so more than 100-obj errors indicates that board was not cleared
    if err > 100-obj: 
        move = 100
        move_acc = (move-err)/move
        cleared = 0
    else:
        move = err+obj
        move_acc = obj/move
        cleared = 1

    return int(move), int(err), move_acc, int(cleared)

In [3]:
# Loop over all the runs
for k in range(len(runs_to_analyze)):
    # Create lists that will hold dataframes for concatenation
    episode_df_list = []
    move_df_list = []
    terminal_results_list = []
    # Form complete run path
    run = runs_to_analyze[k]["config"]
    nickname = runs_to_analyze[k]["nickname"]
    run_path = rule_runs_path+run

    # Get the experiments for that run (each one is a set of trials for a particular rule)
    experiments = [name for name in os.listdir(run_path) if name!=".DS_Store"]
   
    # Loop over the experiments
    for experiment in experiments:
        # Get the neptune id
        neptune_trial = experiment.split("_")[0]
        # String length may vary
        trial_str_length = len(neptune_trial)
        # Rule is everything after the neptune id and an _ character
        rule_name = experiment[trial_str_length+1:]
        
        # Form complete experiment path
        experiment_path = os.path.join(run_path,experiment)
        # Get list of all trials
        trials = [name for name in os.listdir(experiment_path) if name!=".DS_Store"]
        # Sanity check the count (at least for the case where 56 is standard)
        if not(len(trials)==56):
            print(run, experiment, len(trials))

        # Loop over all the trials
        for trial in trials:
            trial_path = os.path.join(experiment_path,trial)
            # Get parameters
            yaml_path = os.path.join(trial_path,"data.yaml")
            with open(yaml_path, 'r') as param_file:
                args = yaml.load(param_file, Loader=SafeLoaderIgnoreUnknown)
                featurization =args["FEATURIZATION"]
                obj_count = args["INIT_OBJ_COUNT"]

            # Import episodic data and process relevant columns
            episodic_data_path = os.path.join(trial_path,"episode_data.csv")
            trial_results = pd.read_csv(episodic_data_path,index_col=0)

            # Episode processing
            # Old
            # Formatting: nickname,rule_name,trial_id,board_id,number_of_pieces,number_of_moves,move_acc,if_clear
            # New
            # Formatting: number_of_moves,number_of_errors,if_cleared
            # w/ header #.nickname,rule_name,trial_id
            trial_results['nickname'] = nickname
            trial_results["rule_name"]=rule_name
            trial_results['trial_id']=trial
            trial_results['board_id']=trial_results['episode']
            trial_results['number_of_pieces']=obj_count
            trial_results[['number_of_moves','number_of_errors','move_acc','if_clear']]=trial_results.apply(move_calculator,args=(obj_count,),axis=1,result_type='expand')
            
            # Append results to a list for concatenation
            episode_df_list.append(trial_results)

    # Concatenate everything into one dataframe
    episode_results_df = pd.concat(episode_df_list,ignore_index=True)
    episode_results_df.sort_values(by=["rule_name","trial_id","board_id"],inplace=True,ignore_index=True)
    # Drop the original columns to arrive at final formatting
    #episode_results_df.drop(labels=['episode','reward','nickname','rule_name','trial_id','board_id','number_of_pieces','move_acc'],axis=1,inplace=True)
    result_path = 'mlc_proc_output/'+nickname
    if not(os.path.exists(result_path)):
        os.mkdir(result_path)
    
    header = "#.nickname,rule_name,trial_id\n#number_of_moves,number_of_errors,if_clear\n"
    result_file = result_path+'/results.csv'
    with open(result_file, 'w') as fp:
        fp.write(header)

    #episode_results_df.to_csv(result_path+'/results.csv',header=["."+nickname,rule_name,"0"],index=False)
    # Write out new episodic format
    for rule in episode_results_df.rule_name.unique():
        for trial in episode_results_df.query("rule_name==@rule").trial_id.unique():
            temp=episode_results_df.query("rule_name==@rule and trial_id==@trial").copy()
            temp.drop(labels=['episode','reward','nickname','rule_name','trial_id','board_id','number_of_pieces','move_acc'],axis=1,inplace=True)
            temp=temp.astype('int')
            temp.to_csv(result_file,header=['.'+nickname,rule,trial],mode='a',index=False)